<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/MultipartiteRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install matplotlib
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-jnqh4aeh
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-jnqh4aeh
     |████████████████████████████████| 235 kB 7.2 MB/s 
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160276 sha256=6463cce807fb0c8870fbbdfd1f0d62e1c91ec836d0b49a04a35319846a025dad
  Stored in directory: /tmp/pip-ephem-wheel-cache-8ylnapbl/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
Successfully built pke
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-07-26 12:02:42.792668: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/PaperA3/Corpus/corpus_BOC_NegCUI.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.dropna()
        

,label,term
0,157267,"C0002893,C0018802,C0149531,C0020295,C0027270,C..."
1,123103,"C0002893,C0007012,C0027270,C0032326,C0013604,C..."
2,139931,"C0262613,C0007012,C0441713,C0020295,C0013604,C..."
3,198295,"C0036690,C0018802,C0024117,C0032326,C1704493,C..."
4,177962,"C0080310,C0027270,C0032326,C0020676,C0392689,C..."
...,...,...
26382,109419.0,"C0036690,C0027270,C0020295,C0018021,C0013604,C..."
26383,160393.0,"C0036690,C0700374,C0020433,C0031256,C0011849,C..."
26384,190986.0,"C0035508,C4054315,C0005367,C0332137,C0010520,C..."
26385,130661.0,"C0232694,C0001883,C0030054,C0700374,C0015081,C..."


In [ ]:
dd=pd.read_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Dictionary.csv')
dictionary = dict(zip(dd.first_cuid, dd.canonical_name))

In [ ]:
import ast
def complex_function( x):
    res =x.split(",")
    tms_text=''    
    for i in range(len(res)):
        try:  
            if(res[i].upper().startswith("NOT")):
              tms_text=tms_text +'NOT '+dictionary[res[i][4:].upper()] +', '
            else:
              tms_text=tms_text +dictionary[res[i].upper()] +', '  
        except :
              print(res[i])    
    return tms_text 

In [ ]:
df=df.dropna()
df['Text'] = df.apply(lambda x: complex_function(x['term']), axis=1)
df.shape

(26357, 3)

In [ ]:
import pke

# initialize a TopicRank keyphrase extraction model
extractor = pke.unsupervised.MultipartiteRank()

### step-2: what we need now is a sample document

In [ ]:
# sample document (2040.abstr from the Inspec dataset)
data_all_new= []
for j in range(len(df)):
    print(len(df)-j)
    sample = df.iloc[j].Text.replace("\n", " ")
    extractor.load_document(input=sample, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=20, stemming=False)
    keyphrases
    # for each of the best candidates
    ls=[]
    for i, (candidate, score) in enumerate(keyphrases):
        ls.append(candidate)
        # print out the its rank, phrase and score
      #print("rank {}: {} ({})".format(i, candidate, score))
    str_tmp = ', '.join([str(elem) for elem in ls])
    data_all_new.append( [df.iloc[j].label,str_tmp])
    if (j % 1000 ==0):
          filename=int(j/1000+1)
          pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/MultipartiteRank/MultipartiteRank_20_'+str(filename)+'_'+str(j)+'.csv')
          data_all_new= []



In [ ]:
pd.DataFrame(data_all_new,columns=['label','Text']).to_csv('/content/drive/MyDrive/PaperA3/MultipartiteRank/MultipartiteRank_20_'+'_'+str(28)+'.csv')
